## 1. Import libraries

In [1]:
import sys
sys.path.append('../')

%matplotlib inline
from collections import defaultdict
from typing import List

import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

import gym

import src.agents as agents
import src.episode as episode
import src.environment as environment
import src.aux_plots as aux_plots

## 2. Auxiliary functions

### 2.1. Reduce state space

In [2]:
def reduce_state(ob):
    # Doesn't matter where we were hit
    ob[16] = 1 if ob[16] != 255 else 0

    # Reduce chicken y-position
    ob[14] = ob[14] // 3

    for b in range(108, 118):
        # The chicken is in the x-posistion ~49
        if ob[b] < 20 or ob[b] > 80:
            # We don't need to represent cars far from the chicken
            ob[b] = 0
        else:
            # Reduce the cars x-positions sample space
            ob[b] = ob[b] // 3

    return ob

### 2.2. Reward policy

In [3]:
def reward_policy(reward, ob, action):
    if reward == 1:
        reward = reward_policy.REWARD_IF_CROSS
    elif ob[16] == 1:  # Collision!
        reward = reward_policy.REWARD_IF_COLISION
    elif action != 1:  # Don't incentivate staying still
        reward = reward_policy.REWARD_IF_STILL

    return reward

### 2.3. Print results

In [4]:
def print_result(i, scores, total_reward, score):
    if i % 10 == 0:
        print(f"Run [{i:4}] - Total reward: {total_reward:7.2f} Mean scores: {sum(scores) / len(scores):.2f} Means Scores[:-10]: {sum(scores[-10:]) / len(scores[-10:]):5.2f} Score: {score:2} ")

## 3. Running algorithms

### 3.1. Hyperparameters

In [5]:
RAM_mask = [
      14  # Chicken Y
    , 16  # Chicken Lane Collide
    , 108, 109, 110, 111, 112, 113, 114, 115, 116, 117  # Car X Coords
]

GAMMA = 0.99
AVAILABLE_ACTIONS = 2
N0 = 2.5

reward_policy.REWARD_IF_CROSS = 50
reward_policy.REWARD_IF_COLISION = -1
reward_policy.REWARD_IF_STILL = -0.1

### 3.2. Monte Carlo

In [ ]:
env, initial_state = environment.get_env()

mc_agent = agents.MonteCarloControl(gamma=GAMMA, available_actions=AVAILABLE_ACTIONS, N0=N0)

In [ ]:
def MonteCarloES(agent, reduce_s, reward_p, RAM_mask: List[int], render: bool=False):
    epi = episode.generate_episode(env, reduce_state=reduce_s, reward_policy=reward_p, agent=agent, RAM_mask=RAM_mask, render=render)
    return agent.update_policy(epi)

In [ ]:
# %%time
# MonteCarloES(agent=mc_agent,reduce_s=reduce_state, reward_p=reward_policy, RAM_mask=RAM_mask, render=False)

In [ ]:
%%time
scores = []
total_rewards = []


n_runs = 5

for i in range(n_runs):
    render = i % 201 == 200

    score, total_reward = MonteCarloES(agent=mc_agent,reduce_s=reduce_state, reward_p=reward_policy, RAM_mask=RAM_mask, render=render)

    scores.append(score)
    total_rewards.append(total_reward)

    print_result(i, scores, total_reward, score)

### 3.3. Q Learning

In [ ]:
env, initial_state = environment.get_env()
ql_agent = agents.QLearning(gamma=GAMMA, available_actions=AVAILABLE_ACTIONS, N0=N0)

In [ ]:
%%time
scores = []
total_rewards = []

n_runs = 1
render = False
for i in range(n_runs):
#     render = i % 200 == 0

    game_over = False
    state = env.reset()
    state = reduce_state(state)[RAM_mask].data.tobytes()  # Select useful bytes
    action = ql_agent.act(state)
    
    score = 0
    total_reward = 0

    while not game_over:
        if render:
            time.sleep(0.025)
            env.render()

        old_state = state
        ob, reward, game_over, _ = env.step(action)

        ob = reduce_state(ob)
        reward = reward_policy(reward, ob, action)

        total_reward += reward

        if reward == reward_policy.REWARD_IF_CROSS:
            score += 1

        state = ob[RAM_mask].data.tobytes()

        ql_agent.update_Q(old_state, state, action, reward)

        action = ql_agent.act(state)  # Next action

    scores.append(score)
    total_rewards.append(total_reward)

    print_result(i, scores, total_reward, score)

### 3.4. Q Learning Approximator

In [6]:
env, initial_state = environment.get_env()
ql_agent_app = agents.QLearningLinearApprox(gamma=GAMMA, available_actions=AVAILABLE_ACTIONS, N0=N0, weights_length=len(RAM_mask)+1)

In [7]:
%%time
scores = []
total_rewards = []

n_runs = 1
render = False
for i in range(n_runs):
#     render = i % 200 == 0

    game_over = False
    state = env.reset()
    state = reduce_state(state)[RAM_mask].data.tobytes()  # Select useful bytes
    action = ql_agent_app.act(state)
    
    score = 0
    total_reward = 0

    while not game_over:
        if render:
            time.sleep(0.025)
            env.render()

        old_state = state
        ob, reward, game_over, _ = env.step(action)

        ob = reduce_state(ob)
        reward = reward_policy(reward, ob, action)

        total_reward += reward

        if reward == reward_policy.REWARD_IF_CROSS:
            score += 1

        state = ob[RAM_mask].data.tobytes()

        ql_agent_app.update_W(old_state, state, action, reward)

        action = ql_agent_app.act(state)  # Next action

    scores.append(score)
    total_rewards.append(total_reward)

    print_result(i, scores, total_reward, score)

Update:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Update:  [-0.2  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
Update:  [-0.296  0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.   ]
Update:  [-0.87024  0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.      -0.28712]
Update:  [-1.0919184  0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
 -0.28712  ]
Update:  [-1.19278207  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 -0.28712   ]
Update:  [-1.25699861  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 -0.28712   ]
Update:  [-1.44386873  0.          0.          0.          0.          0.
 -0.37374025  0.          0.          0.          0.          0.
 -0.28712   ]
Update:  [ -5.77237378   0.           0.         

Update:  [-4.78435441e+110  8.67183320e+080 -3.36283573e+111 -4.15040421e+111
  7.67776442e+078  1.31842351e+058  7.67767308e+078 -2.72352799e+106
  8.06629940e+097 -2.87036400e+111  0.00000000e+000  0.00000000e+000
 -2.49092374e+107]
Update:  [ 2.99604333e+111  8.67183320e+080  2.09585157e+112  2.59617451e+112
  7.67776442e+078  1.31842351e+058  7.67767308e+078 -2.72352799e+106
  8.06629940e+097  1.79765086e+112  0.00000000e+000  0.00000000e+000
 -2.49092374e+107]
Update:  [-9.38251719e+112  8.67183320e+080 -6.56789991e+113 -8.13155453e+113
  7.67776442e+078  1.31842351e+058  7.67767308e+078 -2.72352799e+106
  8.06629940e+097 -5.62950783e+113 -8.39117199e+113  0.00000000e+000
 -2.49092374e+107]
Update:  [ 6.46961385e+115  8.67183320e+080  4.52872955e+116  5.60699863e+116
  7.67776442e+078  1.31842351e+058  7.67767308e+078 -2.72352799e+106
  8.06629940e+097  3.66580177e+116  5.60673901e+116  0.00000000e+000
 -2.49092374e+107]
Update:  [-5.94428455e+116  8.67183320e+080 -4.16099920e+117

Update:  [-8.03426347e+234 -3.40345013e+198  1.06281206e+145  5.60699863e+116
 -5.50102174e+234  7.84396261e+178 -3.60699828e+215 -5.50526961e+234
  1.50511093e+234  3.90940211e+152  1.23670508e+212 -5.50059362e+234
 -6.80319332e+233]
Update:  [-7.94682849e+235 -3.40345013e+198  1.06281206e+145  5.60699863e+116
 -5.19331357e+235  7.84396261e+178 -3.60699828e+215 -4.83656825e+235
  1.50511093e+234  3.90940211e+152  1.23670508e+212 -5.19327075e+235
 -4.25202040e+234]
Update:  [ 2.76237482e+237 -3.40345013e+198  1.06281206e+145  5.60699863e+116
  1.84262893e+237  7.84396261e+178 -3.60699828e+215  1.44021880e+237
  1.50511093e+234  3.90940211e+152  1.23670508e+212  1.70730350e+237
 -4.25202040e+234]
Update:  [ 5.41302455e+237 -3.40345013e+198  1.06281206e+145  5.60699863e+116
  3.60972875e+237  7.84396261e+178 -3.60699828e+215  2.70243296e+237
  1.50511093e+234  3.90940211e+152  1.23670508e+212  3.22196049e+237
  1.21969395e+236]
Update:  [-2.33433759e+238 -3.40345013e+198  1.06281206e+145

/home/marianna/Documentos/01-semesters-unicamp/02-second-semester-unicamp/03-reinforcement-learning/03-projects-rl/FreewayGame/marianna/src/agents.py:156: RuntimeWarning: invalid value encountered in double_scalars
  self.W = self.W + alpha*(reward + (self.gamma * max_value) - self.getApproximation(old_state, action))*self.createFeature(old_state, action)


Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan n

...

Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan n

Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan n

Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan n

Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan n

Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan nan nan]
Update:  [nan nan nan nan nan nan nan nan nan nan nan n

In [ ]:
np.seterr('warn')

In [ ]:
ql_agent_app.update_W(old_state, state, action, reward)

In [ ]:
old_state, state, action, reward